In [1]:
import numpy as np
import lmdb,os,sys,math
caffe_root = '/gpu1/adnan/caffe/'  # this file is expected to be in {caffe_root}/examples
sys.path.insert(0, caffe_root + 'python')
import caffe

import caffe.proto.caffe_pb2
from  caffe.io import datum_to_array

import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

caffe.set_mode_gpu()

In [2]:
net = caffe.Net(caffe_root + 'examples/scene-regression/model_src/regression_model_deploy.prototxt',
                caffe_root + 'examples/scene-regression/model_src/caffe_models/lenet_models/regression_test_iter_1000.caffemodel',
                caffe.TEST)

In [29]:
#####################################
#test shell
####################################
print np.load('image_mean.npy').reshape(640,640,3).shape

(640, 640, 3)


In [35]:
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape}) # getshape of the imput "data" layer 
transformer.set_transpose('data', (2,0,1)) # change to bgr to rgb
transformer.set_mean('data', np.load('image_mean.npy')) # mean pixel, set mean.npy
#transformer.set_raw_scale('data', 255)  # the model operates on images in [0,255] range instead of [0,1]
#transformer.set_channel_swap('data', (2,1,0))  # the model has channels in BGR order instead of RGB

In [38]:
####################################
# load "images" list 
####################################
images=[]
LMDB_FILE_NAME = "test_data_lmdb"
LMDB_PATH = os.path.join(os.getcwd(),'..','LMDB_612',LMDB_FILE_NAME)
env = lmdb.open(LMDB_PATH, readonly=True, lock=False)

visualize = False
number_of_images = 60
datum = caffe.proto.caffe_pb2.Datum() #datum_pb2.Datum()

with env.begin() as txn:
    cur = txn.cursor()
    for i in xrange(number_of_images):
        if not cur.next():
            cur.first()
        key, value = cur.item()
        datum.ParseFromString(value)
        img_data = datum_to_array(datum)
       
        if visualize:
            plt.imshow(img_data.transpose([1,2,0]))
            plt.show()
        else:
            #print type(img_data)#.shape
            images.append(img_data) # image in BGR

In [39]:
####################################
# load "scores" list 
####################################
scores=[]
LMDB_FILE_NAME = "test_score_lmdb"
LMDB_PATH = os.path.join('..','LMDB_612',LMDB_FILE_NAME)
env = lmdb.open(LMDB_PATH, readonly=True, lock=False)

visualize = False
number_of_images = 60
datum = caffe.proto.caffe_pb2.Datum() #datum_pb2.Datum()

with env.begin() as txn:
    cur = txn.cursor()
    for i in xrange(number_of_images):
        if not cur.next():
            cur.first()
        key, value = cur.item()
        datum.ParseFromString(value)
        score_data = datum_to_array(datum)
        #print score_data.reshape(1)[0]
        scores.append(float(score_data.reshape(1)[0])) # image in BGR

In [40]:
actual=[]
predicted=[]

for img,score in zip(images,scores):
    img = np.swapaxes(img,0,1)
    img = np.swapaxes(img,1,2)
    net.blobs['data'].data[0] = transformer.preprocess('data', img)
    out = net.forward()
    predicted.append(out['ip2'][0][0])
    actual.append(score)


In [56]:
a = np.array(actual,dtype=float)
p = np.array(predicted,dtype=float)

diff= np.array((a - p) ** 2)

print """
MSE : mean square error

     Mean: %12.8f
      STD: %12.8f
      Max: %12.8f
      Min: %12.8f
      
"""%(np.mean(diff),np.std(diff),np.max(diff),np.min(diff))




MSE : mean square error

     Mean:   0.00793423
      STD:   0.01053546
      Max:   0.05218469
      Min:   0.00000017
      

